# Extract video from sequence of images
During training, final time boxplots, timestamps left, and the class probabilities are extracted and displayed as graphs once every 5 epochs. From those sequences of images, a video is created. This notebook shows how to extract the video from the sequence of images.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os 
os.environ['MPLCONFIGDIR'] = '/myhome'
from datetime import datetime
# sys.path.append(os.path.abspath(os.path.join(os.path.dirname(__file__), os.path.pardir)))
sys.path.append("..")
import pandas as pd
import wandb
from IPython.display import Image
from IPython.display import Video
from utils.results_analysis.extract_video import download_images, add_files_to_images, save_video


## Download the model from wandb 

In [3]:
api = wandb.Api()
entity, project, sweep = "aurenore", "MasterThesis", "x6ma4kpb"
runs = api.runs(entity + "/" + project)

summary_list, config_list, name_list, sweep_list, start_date_list = [], [], [], [], []
for run in runs:
    # .summary contains output keys/values for
    # metrics such as accuracy.
    #  We call ._json_dict to omit large files
    summary_list.append(run.summary._json_dict)

    # .config contains the hyperparameters.
    #  We remove special values that start with _.
    config_list.append({k: v for k, v in run.config.items() if not k.startswith("_")})

    # .name is the human-readable name of the run.
    name_list.append(run.name)
    
    if run.sweep:
        sweep_list.append(run.sweep.name)
    else:
        sweep_list.append(None)
        
    start_date_list.append(run.createdAt)


runs_df = pd.DataFrame(
    {"summary": summary_list, "config": config_list, "name": name_list, "sweep": sweep_list, "start_date": start_date_list}
)
runs_df

,summary,config,name,sweep,start_date
0,"{'alphas': [0.9911110997200012, 0.004444444552...","{'mu': 150, 'loss': 'daily_reward_lin_regr', '...",splendid-sweep-12,piecewise_lin_regr,2024-06-10T14:27:46
1,"{'earliness_reward': 0, 'classification_loss':...","{'mu': 150, 'loss': 'daily_reward_lin_regr', '...",misunderstood-sweep-11,piecewise_lin_regr,2024-06-10T14:27:26
2,{'conf_mat_table': {'artifact_path': 'wandb-cl...,"{'mu': 150, 'loss': 'daily_reward_lin_regr', '...",firm-sweep-10,piecewise_lin_regr,2024-06-10T14:23:20
3,"{'precision': 0.6903013676664816, 'harmonic_me...","{'mu': 150, 'loss': 'daily_reward_lin_regr', '...",blooming-sweep-9,piecewise_lin_regr,2024-06-10T14:22:22
4,"{'kappa': 0.5830747903693067, '_wandb': {'runt...","{'mu': 150, 'loss': 'daily_reward_lin_regr', '...",different-sweep-8,piecewise_lin_regr,2024-06-10T13:29:09
...,...,...,...,...,...
762,"{'loss': {'testloss': 4.522304058074951, 'trai...","{'alpha': 0.5, 'device': 'cuda', 'epochs': 100...",TempCNN-seqlength150,None,2024-03-28T15:16:29
763,"{'classification_loss': 6.784744739532471, 'cl...","{'alpha': 0.5, 'device': 'cpu', 'epochs': 100,...",LSTM-seqlength150,None,2024-03-28T15:15:39
764,{'classification_earliness': 0.189819240706228...,"{'alpha': 0.5, 'device': 'cuda', 'epochs': 100...",TempCNN-seqlength70,None,2024-03-28T13:54:08
765,"{'_step': 96, 'accuracy': 0.7814184661430005, ...","{'alpha': 0.5, 'device': 'cuda', 'epochs': 100...",ELECTS-LSTM-hiddendims=32,None,2024-03-27T14:55:09


get the runs with certain sweep name, and created after a certain date

In [4]:
sweep_name = "daily_reward_lin_regr_with_mus_v2"
sweep_name = "piecewise_lin_regr"
# get the best run
df = runs_df[runs_df.sweep == sweep_name]


Pick the run with the highest harmonic mean 

In [5]:
# get the run with the highest harmonic mean
metric = "harmonic_mean"
chosen_run_idx = df.summary.apply(lambda x: x[metric]).idxmax()
chosen_run = runs[chosen_run_idx]
chosen_run_config = chosen_run.config
print("chosen run: ", chosen_run.name)
print(f"with {metric}: ", chosen_run.summary._json_dict[metric])

chosen run:  playful-sweep-3
with harmonic_mean:  0.6605583970625466


## Sequence of image to video 

In [6]:
local_path = f"../results_analysis_notebooks/media/{chosen_run.name}"

In [7]:
name_image = "class_probabilities_wrt_time"

download_images(name_image, chosen_run, local_path)
images, images_directory = add_files_to_images(local_path, name_image)
video_path = save_video(images_directory, images)

Downloaded media/images/class_probabilities_wrt_time_100_bd822efb14270ec54dd1.png
Downloaded media/images/class_probabilities_wrt_time_10_c5b1160d7bbb4f5c186c.png
Downloaded media/images/class_probabilities_wrt_time_15_c8abb45fbcbc20707e14.png
Downloaded media/images/class_probabilities_wrt_time_1_412a2a26c7e6621950bc.png
Downloaded media/images/class_probabilities_wrt_time_20_ce7d8d2c84ede10435fe.png
Downloaded media/images/class_probabilities_wrt_time_25_58d97c41eaa7f679de0d.png
Downloaded media/images/class_probabilities_wrt_time_30_e3f2b8c86e13673006df.png
Downloaded media/images/class_probabilities_wrt_time_35_db588ae58494dcb1c29d.png
Downloaded media/images/class_probabilities_wrt_time_40_6bfd0826454eb5bfb635.png
Downloaded media/images/class_probabilities_wrt_time_45_06fe1da6765c4c41f37a.png
Downloaded media/images/class_probabilities_wrt_time_50_1851b850d594302a343e.png
Downloaded media/images/class_probabilities_wrt_time_55_1e3c7a92204b0ca5d562.png
Downloaded media/images/clas

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (1500, 4900) to (1504, 4912) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


21 images added to the video.
Video saved at ../results_analysis_notebooks/media/playful-sweep-3\media\images\class_probability_wrt_time.mp4


In [8]:
# play video in the notebook
Video(video_path)

let's do the same for boxplots

In [9]:
name_image = "boxplot"

download_images(name_image, chosen_run, local_path)
images, images_directory = add_files_to_images(local_path, name_image)
video_path = save_video(images_directory, images, name_image+"_video.mp4")

Video(video_path)


Downloaded media/images/boxplot_100_836ea9ea6f96cf87e23d.png
Downloaded media/images/boxplot_10_7eaede13e809fa47fdc4.png
Downloaded media/images/boxplot_15_cac13860861ea60fbc3b.png
Downloaded media/images/boxplot_1_f1e85819b24134accf39.png
Downloaded media/images/boxplot_20_36321afb90e07f91c2b9.png
Downloaded media/images/boxplot_25_5307acc7c5b3894ff12c.png
Downloaded media/images/boxplot_30_227f9943bbb6702cb237.png
Downloaded media/images/boxplot_35_70203939845d81cd9233.png
Downloaded media/images/boxplot_40_92a4ce40ebc042fa1aa8.png
Downloaded media/images/boxplot_45_61a998be845d58d5628a.png
Downloaded media/images/boxplot_50_6e63d728fb4792e2bc9e.png
Downloaded media/images/boxplot_55_961effd9bcbc4ceb5665.png
Downloaded media/images/boxplot_5_80e0dd66094eaa640a52.png
Downloaded media/images/boxplot_60_535ba825cb15b5bea7f3.png
Downloaded media/images/boxplot_65_1558c87dbe7305af5c34.png
Downloaded media/images/boxplot_70_ef35593cbccab8776d28.png
Downloaded media/images/boxplot_75_e7e8a8

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (1500, 700) to (1504, 704) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


21 images added to the video.
Video saved at ../results_analysis_notebooks/media/playful-sweep-3\media\images\boxplot_video.mp4


and for timestamps left

In [10]:
name_image = "timestamps_left_plot"

download_images(name_image, chosen_run, local_path)
images, images_directory = add_files_to_images(local_path, name_image)
video_path = save_video(images_directory, images, name_image+"_video.mp4")

Video(video_path)

Downloaded media/images/timestamps_left_plot_100_1b00f31039ed0fdbfb83.png
Downloaded media/images/timestamps_left_plot_10_81f98435c56ffb2d7b20.png
Downloaded media/images/timestamps_left_plot_15_a5ee012714da29976ee0.png
Downloaded media/images/timestamps_left_plot_1_7f3e7485263e52e36571.png
Downloaded media/images/timestamps_left_plot_20_5a525077e66655e584e4.png
Downloaded media/images/timestamps_left_plot_25_dec76ac833e0e54b85e8.png
Downloaded media/images/timestamps_left_plot_30_185d41e4974c81fc2e1a.png
Downloaded media/images/timestamps_left_plot_35_1e39fad9c9c00b12d23c.png
Downloaded media/images/timestamps_left_plot_40_63fea8add696c68071cc.png
Downloaded media/images/timestamps_left_plot_45_400bc15e3b516f67d396.png
Downloaded media/images/timestamps_left_plot_50_6c436330f41fd32182a1.png
Downloaded media/images/timestamps_left_plot_55_b4c682f5ca2f931c0d07.png
Downloaded media/images/timestamps_left_plot_5_60e44ac42aef800ce409.png
Downloaded media/images/timestamps_left_plot_60_475a

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (1500, 700) to (1504, 704) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


21 images added to the video.
Video saved at ../results_analysis_notebooks/media/playful-sweep-3\media\images\timestamps_left_plot_video.mp4
